In [ ]:
from ultralytics import YOLO
from training_utils import PrepareDataset
import os

### Some constants

In [ ]:
dataset_yaml_path = "/training/ultralytics/ultralytics/cfg/datasets/verdant.yaml"
coco_classes_file = "/dataset/classes.txt"
runs_directory = "/training/runs"
training_task = "pose"   # 'detect' for bbox | 'pose' for hybrid model 
experiment_name = "exp"  # Results will be saved with this name  under runs/<task>/<exp_name>

In [ ]:
# Some utility functions
def GetModelYaml(task):
    if task == "detect":
        return "yolov8n.yaml"
    elif task == "pose":
        return "yolov8n-pose.yaml"
    print(f"Unknown task {task}") 
    return None
        
def GiveModel(ckpt_path):
    if os.path.exists(ckpt_path):
        return YOLO(ckpt_path)
    print(f"[ERROR] : Model {ckpt_path} does not exists")         
    return None

def LoadBestModel():
    best_ckpt = f"{runs_directory}/{training_task}/{experiment_name}/weights/best.pt"
    return GiveModel(best_ckpt)


### Prepare dataset.yaml file
Our dataset is present at `/dataset`. Read `classes.txt` file and fill the `verdant.yaml` file. Ultralytics dataloader will load this file.

In [ ]:
# Prepare the dataset yaml file
PrepareDataset(coco_classes_file, dataset_yaml_path, training_task)
!cat /training/ultralytics/ultralytics/cfg/datasets/verdant.yaml  # To check the file

### Train the model

In [ ]:
model = YOLO(GetModelYaml(training_task))  # Initialize model

In [ ]:
# To get a pretrained model
# !/training/get_model_candidate.sh <model_hash>    # Replace <model_hash> with the hash of the model you want to download. This will print the path of the downloaded model
# model.load(<path_to_the_model>)                   # Load the model

In [ ]:
model.train(
    task=training_task,
    data="verdant.yaml", 
    epochs=1,
    flipud=0.5, 
    fliplr=0.5, 
    scale=0.2, 
    mosaic=0.0,                                 # Please set this to 0.0 TODO: Fix the issue with mosaic and keypoint detection 
    imgsz=768, 
    seed=1, 
    batch=8,
    name=experiment_name,
    device=[0],
    ) 

### Evaluation

In [ ]:
# model = LoadBestModel()    # To load the best model
metrics = model.val(data="verdant.yaml")

### Export the model

In [ ]:
model = LoadBestModel()    # To load the best model
path = model.export(format="onnx", imgsz=[768,768], opset=12)
os.system(f"mv {path} best.onnx")

path = model.export(format="onnx", imgsz=[2144,768], opset=12)
os.system(f"mv {path} best_full_height.onnx")